### Practice: Parameter Efficient Fine-Tuning
In this notebook, you're gonna fine-tune large language models within limited GPU memory.

In [2]:
%pip install --quiet transformers==4.34.1 accelerate==0.24.0 sentencepiece==0.1.99 optimum==1.13.2 peft==0.5.0 bitsandbytes==0.41.2.post2

import torch
import torch.nn as nn
import torch.nn.functional as F

import transformers
from tqdm.auto import tqdm, trange
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Note: you may need to restart the kernel to use updated packages.


In [3]:
model_name = 'Enoch/llama-7b-hf'

# loading Llama tokenizer ...
tokenizer = transformers.LlamaTokenizer.from_pretrained(model_name, device_map=device)
tokenizer.pad_token_id = tokenizer.eos_token_id

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:


# ... and the model itself
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name, device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
    load_in_4bit=True, torch_dtype=torch.float32,  # weights are 4-bit; layernorms and activations are fp32
)
for param in model.parameters():
    param.requires_grad=False

model.gradient_checkpointing_enable()  # only store a small subset of activations, re-compute the rest.
model.enable_input_require_grads()     # override an implementation quirk in gradient checkpoints that disables backprop unless inputs require grad
# more on gradient checkpointing: https://pytorch.org/docs/stable/checkpoint.html https://arxiv.org/abs/1604.06174

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

### Prompt tuning: the story of a fox (2 pts)

![img](https://i.imgur.com/Ux3qQAu.png) (source: theodd1souts.fandom.com)

In [ ]:
prompt = 'A quick brown fox'
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)

for i in range(10):
    out = model(**batch)
    # print(batch['input_ids'].shape, out.logits.shape)
    next_token = out.logits[0, -1].argmax(-1).reshape(1, 1)
    # print(next_token)
    # print(tokenizer.decode(out.logits[0, 4:].argmax(-1)))
    batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
    batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

print("\nOutput:", tokenizer.decode(batch['input_ids'][0].cpu().numpy().tolist()))



Output: <s>A quick brown fox jumps over the lazy dog.

A quick


What a blatant lie! This particular fox assures you that it didn't in fact jump over the lazy dog. No, sir! The fox was just minding its own business. __Your task is to train the model to say truth: no dog was jumped over today.__

In [ ]:
the_truth = "A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it anyway!"
batch = tokenizer(the_truth, return_tensors='pt', return_token_type_ids=False).to(device)
outputs = model(**batch)

next_word_logits = outputs.logits[:, :-1]
true_next_tokens = batch['input_ids'][:, 1:]
loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))

print("Loss:", loss)

Loss: tensor(3.0725, device='cuda:0', grad_fn=<NllLossBackward0>)


Except, we can't train the entire model - that would be 28GB gradients in float32. Instead, let's run [prompt tuning](https://arxiv.org/abs/2104.08691).

![img](https://i.imgur.com/VwNNKnb.png)


In [ ]:
class WordEmbeddingsWithLearnedPrompts(nn.Module):
    """
    To perform prompt tuning, you will need to replace model's original word embeddings with a layer - THIS layer
     - that inserts trainable prompts instead of the first N token embeddings. """

    def __init__(self, word_embeddings: nn.Embedding, num_prompts: int):
        super().__init__()
        self.original_word_embeddings = word_embeddings
        self.num_prompts = num_prompts
        self.learnable_prompts = nn.Parameter(
            torch.randn(1, num_prompts, word_embeddings.embedding_dim), requires_grad=True)

    def forward(self, input_ids: torch.LongTensor):
        # input_ids shape: [batch_size, seq length]
        assert input_ids.dtype == torch.int64
        assert input_ids.shape[1] > self.num_prompts
        assert torch.all(input_ids[:, :self.num_prompts] == tokenizer.pad_token_id).item(), "don't forget to prepend several BOS tokens to input_ids"

        # Your task: embed input_ids, but replace the first :num_prompts: tokens with self.learnable_prompts
        # This is because we will prepend :num_prompts: padding tokens at the beginning

        # After you are done, you must produce a word embedding vector for each token in input_ids,
        # except that the first :num_prompts: vectors should equal learnable_prompts;
        # any additional vectors after first :num_prompts: ones should be embedded as usual
        # Note: since you're dealing with trainable params, please torch.cat instead of item assignment

        output = self.original_word_embeddings(input_ids[:, self.num_prompts:])
        your_outputs_with_prompts_as_per_instructions_above = torch.cat([self.learnable_prompts, output], axis=1)
        return your_outputs_with_prompts_as_per_instructions_above

        return

In [ ]:
num_prompts = 16
test_emb_layer = WordEmbeddingsWithLearnedPrompts(model.model.embed_tokens, num_prompts=num_prompts).to(device)
test_input_ids = tokenizer("a cat say on a may", return_tensors='pt')['input_ids'].to(device)

space_for_prompts = torch.full([len(test_input_ids), num_prompts], fill_value=tokenizer.pad_token_id,
                               dtype=torch.int64, device=device)
test_inputs_with_prompts = torch.cat([space_for_prompts, test_input_ids], dim=1)

with torch.cuda.amp.autocast():
  test_prompt_embeddings = test_emb_layer(test_inputs_with_prompts)

print(test_prompt_embeddings.shape[:2])
print(test_inputs_with_prompts.shape)
assert test_prompt_embeddings.shape[:2] == test_inputs_with_prompts.shape
assert test_prompt_embeddings.shape[-1] == model.config.hidden_size
assert torch.allclose(test_prompt_embeddings[:, :num_prompts], test_emb_layer.learnable_prompts.float())
assert torch.allclose(test_prompt_embeddings[:, num_prompts:], model.model.embed_tokens(test_input_ids).float())
print("Looks legit!")

torch.Size([1, 23])

torch.Size([1, 23])

Looks legit!


__Now that it works,__ let's inject learnable prompts into the main model and teach it about foxes.

In [ ]:
assert isinstance(model.model.embed_tokens, nn.Embedding), "you have already replaced the embedding layer. If the replacement is broken, please reload the model"

model.model.embed_tokens = WordEmbeddingsWithLearnedPrompts(model.model.embed_tokens, num_prompts=num_prompts).to(device)

opt = torch.optim.Adam([model.model.embed_tokens.learnable_prompts], lr=0.01)

In [ ]:
while loss.item() > 0.1:
    the_truth = "A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it anyway!"
    batch = tokenizer(the_truth, return_tensors='pt', return_token_type_ids=False).to(device)
    space_for_prompts = torch.full([len(test_input_ids), num_prompts], fill_value=tokenizer.pad_token_id,
                                dtype=torch.int64, device=device)
    # print('Space for prompts:', space_for_prompts.shape)
    # print('Input ids before cat:', batch['input_ids'].shape)
    batch['input_ids'] = torch.cat([space_for_prompts, batch['input_ids']], dim=1)
    batch['attention_mask'] = torch.cat([torch.ones_like(space_for_prompts), batch['attention_mask']], dim=1)
    # print('Input ids after cat:', batch['input_ids'].shape)
    outputs = model(**batch)
    # print('Output Logits:', outputs.logits.shape)
    # print('True Next Tokens Before:', batch['input_ids'].shape)

    next_word_logits = outputs.logits[:, num_prompts : -1, :]
    true_next_tokens = batch['input_ids'][:, num_prompts + 1:]
    # print('Next Word Logits:', next_word_logits.shape, next_word_logits.argmax(dim=2))
    # print('True Next Tokens:', true_next_tokens.shape, true_next_tokens)
    loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))
    print("Loss:", loss)
    loss.backward()
    opt.step()
    opt.zero_grad()


# raise ValueError("Your task: iteratively train the model to reduce loss using prompt optimizer (opt)")


assert loss.item() <= 0.1
print("Good job!")

Loss: tensor(7.3237, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(6.9157, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(6.4277, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(5.9162, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(5.4310, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(5.0175, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(4.6749, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(4.3940, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(4.1556, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(3.9375, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(3.7254, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(3.5132, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(3.3014, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(3.0933, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(2.8930, device='cuda:0', grad_fn=<NllLossBackward

In [ ]:
prompt = 'A quick brown fox'
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
batch['input_ids'] = torch.cat([space_for_prompts, batch['input_ids']], dim=1)
batch['attention_mask'] = torch.cat([torch.ones_like(space_for_prompts), batch['attention_mask']], dim=1)


for i in range(15):
    next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
    batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
    batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

print("\nOutput:", tokenizer.decode(batch['input_ids'][0, num_prompts:].cpu().numpy().tolist()))

# if you did everything right, the model will deny that the fox jumped over the lazy dog



Output: <s>A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it


### Using HuggingFace PEFT (2 points)

[`peft`](https://huggingface.co/docs/peft/index) is a transformer's sister library that allows you to apply various __p__arameter __e__fficient __f__ine-__t__uning methods to pre-trained transformers. The library imlements both prompt tuning, prefix tuning, as well as several adapter-based techniques under a common interface:



In [ ]:
import peft
assert isinstance(model.model.embed_tokens, nn.Embedding), "please reload the model"
num_virtual_tokens = 16

peft_config = peft.PromptTuningConfig(task_type=peft.TaskType.CAUSAL_LM, num_virtual_tokens=num_virtual_tokens)
model = peft.get_peft_model(model, peft_config)  # note: for most peft methods, this line also modifies model in-place
print("Trainable parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))
print("Total parameters (excluding quantization):", sum(p.numel() for p in model.parameters()))

Trainable parameters: 65536

Total parameters (excluding quantization): 3500478464


In [ ]:
# Your task: optimize the PEFT-wrapped model to achieve next token prediction loss < 0.1, but this time using PEFT
# Please note: you no longer need to prepend PAD tokens, but you still need to skip :num_virtual_tokens: first logits.
# Finally, generate the sentence to make sure that the model learned the truth.

In [ ]:
# Feel free to structure your code as you see fit - as long as it's legible :)
def test_model_output():
    prompt = 'A quick brown fox'
    batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
    # batch['input_ids'] = torch.cat([space_for_prompts, batch['input_ids']], dim=1)
    # batch['attention_mask'] = torch.cat([torch.ones_like(space_for_prompts), batch['attention_mask']], dim=1)


    for i in range(15):
        next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
        batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
        batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

    print("\nOutput:", tokenizer.decode(batch['input_ids'][0, :].cpu().numpy().tolist()))
test_model_output()



Output: <s>A quick brown fox

              


In [ ]:
opt = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
def train_model_output_until_convergence():
    loss = torch.tensor(100)
    while loss.item() > 0.1:
        the_truth = "A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it anyway!"
        tokenized_truth = tokenizer(the_truth, return_tensors='pt', return_token_type_ids=False).to(device)

        out = model(**tokenized_truth)
        next_word_logits = out.logits[:, num_virtual_tokens:-1]
        true_next_tokens = tokenized_truth['input_ids'][:, 1:]
        # print(tokenizer.decode(out.logits[0, :].argmax(-1)))
        # print(next_word_logits.flatten(0, 1).shape)
        # print(true_next_tokens.flatten(0, 1).shape)
        loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))
        print("Loss:", loss)
        loss.backward()
        opt.step()
        opt.zero_grad()

In [ ]:
train_model_output_until_convergence()
test_model_output()

Loss: tensor(7.1602, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(6.4531, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(5.5564, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(4.9608, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(4.2116, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(3.7268, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(3.4428, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(3.2170, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(3.0232, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(2.8944, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(2.7871, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(2.6773, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(2.5724, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(2.4801, device='cuda:0', grad_fn=<NllLossBackward0>)

Loss: tensor(2.3982, device='cuda:0', grad_fn=<NllLossBackward

In [ ]:
test_model_output()



Output: <s>A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it


### Parameter-efficient finetuning with LoRA (2 points)

When training on more serious tasks, you can use low-rank adapters based on the [LoRA paper](https://arxiv.org/pdf/2106.09685.pdf).

The core idea is to add low-rank adapters __in parallel with existing linear layers,__ like this:
<center><img src="https://i.imgur.com/6bQLNiG.png" width=240px></center>

In the original LoRA paper, the adapters were only added to attention projection matrices. However, [subsequent works](https://arxiv.org/abs/2305.14314) show that it is useful to adapt FFNs as well. But before we do any training, we need to implement the basic LoRA layer.

In [1]:
import os
os.environ["WANDB_DISABLED"] = "True"

In [4]:
# re-load the model to remove any previous PEFT tuners
model_name = 'Enoch/llama-7b-hf'
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name, device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
    load_in_4bit=True, torch_dtype=torch.float32,  # weights are 4-bit; layernorms and activations are fp32
)
for param in model.parameters():
    param.requires_grad=False
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [5]:
class LoRALayer(nn.Module):
    """Wraps a linear layer with LoRA-like adapter. Wraps an existing OPT linear layer"""
    def __init__(self, module: nn.Linear, rank: int):
        super().__init__()
        self.module = module  # pre-trained (frozen) linear layer
        self.adapter_A = nn.Parameter(torch.empty(module.in_features, rank, device=module.weight.device))
        nn.init.kaiming_uniform_(self.adapter_A, a=5 ** 0.5)
        self.adapter_B = nn.Parameter(torch.zeros(rank, module.out_features, device=module.weight.device))

    def forward(self, input):
        # Apply self.module and LoRA adapter, return the sum (self.module outputs + adapter outputs)
        module_output = self.module(input)
        adapter_output = (input @ self.adapter_A) @ self.adapter_B
        return module_output + adapter_output

In [6]:
# test your implementation
test_linear = nn.Linear(128, 128)
test_linear.weight.data[...] = torch.eye(128)
test_adapter = LoRALayer(test_linear, rank=8)

assert torch.allclose(test_adapter(torch.ones(1, 1, 128)), test_linear.bias + 1), "please check your forward pass"

test_adapter.adapter_A.data[...] = torch.linspace(0.1, -0.5, 128 * 8).view(128, 8)
test_adapter.adapter_B.data[...] = torch.linspace(0.5, -0.1, 128 * 8).view(8, 128)
test_linear.bias.data[...] = torch.linspace(1., -1., 128)

dummy_loss = F.mse_loss(test_adapter(torch.ones(1, 128) / 128).squeeze(), torch.linspace(-1, 1, 128))
assert torch.allclose(dummy_loss, torch.tensor(1.3711389), rtol=0, atol=1e-4)
dummy_loss.backward()
assert all(w.grad is not None for w in [test_adapter.adapter_A, test_adapter.adapter_B]), "some adapter weights have no grad"
assert torch.allclose(test_adapter.adapter_A.grad.sum(), torch.tensor(-0.60158), rtol=0, atol=1e-4), "bad grad w.r.t. A"
assert torch.allclose(test_adapter.adapter_B.grad.sum(), torch.tensor(0.9931), rtol=0, atol=1e-4), "bad grad w.r.t. B"
# note: bad grad means that your code is different from LoRA paper OR that your code is not autograd-friendly (e.g. no_grad)
del dummy_loss, test_linear, test_adapter
print("All tests passed!")

All tests passed!


### Apply LoRA to the model

The code below applies LoRA adapters on top of Q/K/V linear layers in Llama attention. You may also choose to modify other layers:
* self_attn.o_proj - attention output projection
* mlp.up_proj, mlp.gate_proj, mlp.down_proj - transformer feedforward layers
* lm_head - output LM head

__Note:__ please scroll down for the homework task

In [7]:
lora_rank = 8

for name, module in model.model.layers.named_modules():
    if 'LlamaDecoderLayer' in repr(type(module)):
        module.self_attn.q_proj = LoRALayer(module.self_attn.q_proj, rank=lora_rank).to(device)
        module.self_attn.k_proj = LoRALayer(module.self_attn.k_proj, rank=lora_rank).to(device)
        module.self_attn.v_proj = LoRALayer(module.self_attn.v_proj, rank=lora_rank).to(device)

assert sum(isinstance(module, LoRALayer) for module in model.modules()) == 96  # for Llama-7B

In [8]:
# tokenizer = transformers.LlamaTokenizer.from_pretrained(model_name, device_map=device)
# tokenizer.pad_token_id = tokenizer.eos_token_id

In [9]:
batch = tokenizer("This model wants to share its greatest secret:", return_tensors='pt', return_token_type_ids=False)
# test a single training step, make sure we get meaningful gradients
with torch.cuda.amp.autocast(dtype=torch.float32):
    out = model.forward(**batch)
    (out.logits.norm() / 100).backward()

for i, module in enumerate(model.modules()):
    if isinstance(module, LoRALayer):
        assert module.adapter_B.grad is not None
        assert module.adapter_B.grad.norm().item() > 0

model.zero_grad(set_to_none=True)
print("Grad check successful, well done!")

Grad check successful, well done!


### (example) How to train your model

The example below shows how to train the LoRA adapters on a dummy dataset. You will need to run a _similar_ training task later.

__Note:__ please scroll down for the homework task

In [10]:
# checking if the model can learn. Change max_steps for proper training
import datasets
data = datasets.load_dataset("Abirate/english_quotes", split="train[:32]") # 32 lines
data = data.map(lambda samples: tokenizer(samples['quote']), batched=True)
model._hf_peft_config_loaded = True  # silence a warning from HF trainer

trainer = transformers.Trainer(
    model=model, train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2, gradient_accumulation_steps=1,
        # note: if you want larger batch size, increase gradient_accumulation_steps
        warmup_steps=250, max_steps=5, learning_rate=2e-4, fp16=True,
        logging_steps=1, output_dir='outputs', report_to=None),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
# if you see cache warnings, set `model.config.use_cache = False` to silence them. Please re-enable for inference!

trainer.train()

# NOTE: this is just an example! you do not have to wait for this progressbar to finish :)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/Abirate--english_quotes-7ef692ccb59fbf2a/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?ba/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
1,1.891600
2,1.695600
3,0.894100
4,1.745500
5,1.174600


TrainOutput(global_step=5, training_loss=1.4802877187728882, metrics={'train_runtime': 6.0433, 'train_samples_per_second': 1.655, 'train_steps_per_second': 0.827, 'total_flos': 12301361233920.0, 'train_loss': 1.4802877187728882, 'epoch': 0.31})

### Final task: *actually* train the model (4 points)

Your task is to fine-tune the model to _generate python code_. Please use the above examples for inspiration. More specifically,

* __dataset:__ use [codeparrot-clean](https://huggingface.co/datasets/codeparrot/codeparrot-clean) or any other data containing python code. Since you do not need much data for this excercise, it is enough to use just shorter validation subset of `codeparrots`
* __preprocessing:__ select python code based on file extentions (.py)  (may skip in case of codeparrot - it is 100% python)
* __short lines:__ please take the first 512 characters of each line
* __adapter type:__ please use LoRA as defined above __plus at least one of:__
   - extra adapter on lm_head
   - extra adapter on MLP components (mlp.*)
   - trainable input embeddings (requires tweaking memory usage)

* __training:__ you do not have to train to convergence. If all goes well, your model should `.generate` code after 500 steps. Please use batch size of at least 4 (4 x 1 x 512 tokens) using `gradient_accumulation_steps=4`.


Note: the peft library also has LoRA implementation. However, we ask that for this assignment you show at least one complete training run with your own LoRA code.

__Alternative assignment:__ Instead of doing python code, feel free to substitute the task with any other dataset, e.g. your favorite artist or podcast, as long as it's ethical. If you choose your own task, please show examples of what your model learned - or did not learn, akin to the code examples below.

In [11]:
prompts =  ['', 'import', 'from', 'while', 'try', 'if', 'for', 'torch', "include"]  # feel free to add a few more that are not 100% assiciated with Python

# <A WHOLE LOT OF YOUR CODE>
# generate baseline samples with the selected prompts before finetuning
# please feel free to use transformers.Trainer (as above) or your custom training code
# after the training concludes, please show examples of text generated by your model. It is expected to look like Python code fragments
# print the generation examples nicely (suggestion: use pandas or HTML) for easier comparison
# note: your LoRA-enhanced model can run generation the same way as the non-trained model (above)

In [12]:
# Feel free to structure your code as you see fit - as long as it's legible :)
def test_model_output(prompt, n=64):
    model.eval()
    batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
    for i in range(n):
        with torch.no_grad():
            next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
        batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
        batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)
    output = tokenizer.decode(batch['input_ids'][0, :].cpu().numpy().tolist())
    print("\nOutput:", output)
    model.train()
    return output

In [13]:
print('Outputs before training')
before_outputs = []
for prompt in prompts:
    output = test_model_output(prompt)
    before_outputs.append(output)
    print('----------------')

Outputs before training

Output: <s>▶▶ 2019-2020 ◀◀
The 2019-2020 school year is here! We are so excited to see what God has in store for us this year. We are looking forward to another year of growth and learning.
----------------

Output: <s> import Foundation

public extension NSURL {
    public var absoluteString: String {
        return String(cString: CFBundleGetBundleWithURL(self).UTF8String)
    }
}</s><s>package com.google.gerrit.server.restapi;

import static javax.
----------------

Output: <s>from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys

from absl import flags

from tensorflow.python import pywrap_tensorflow
from tensorflow.python.eager import context
from
----------------

Output: <s>while(1)
while(1) {
    // do something
}
\end{code}

Comment: This is not the same as the OP's code.

Comment: @Jeffrey: It's the same as the OP's code, except that it's
----------------

Output: <s>try to f

In [14]:
import datasets
ds = datasets.load_dataset('codeparrot/codeparrot-valid-v2-near-dedup')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/codeparrot--codeparrot-valid-v2-near-dedup-b43bd121f4d256c1/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [30]:
ds = ds.shuffle()

In [29]:
# ds['train'][164]['content'].split('\n')

In [16]:
def make_short_lines(samples):
    content = samples['content']
    # take only lines less than 512 symbols and no more than 50 lines (avoid GPU crash)
    new_content = ["\n".join([x[:512] for x in con.split("\n")[0:50]]) for con in content]
    return tokenizer(new_content)

In [17]:
ds = ds.map(make_short_lines, batched=True)

  0%|          | 0/46 [00:00<?, ?ba/s]

In [18]:
for name, module in model.model.layers.named_modules():
    if 'LlamaDecoderLayer' in repr(type(module)):
#         print([x for x in dir(module) if 'mlp' in x])
#         print(vars(module.mlp))
        module.mlp.up_proj = LoRALayer(module.mlp.up_proj, rank=lora_rank).to(device)
        module.mlp.gate_proj = LoRALayer(module.mlp.gate_proj, rank=lora_rank).to(device)
        module.mlp.down_proj = LoRALayer(module.mlp.down_proj, rank=lora_rank).to(device)

In [19]:
sum(isinstance(module, LoRALayer) for module in model.modules())

192

In [20]:
# torch.save(model.state_dict(), 'model_lora.pt')

In [21]:
# model.load_state_dict(torch.load('model_lora.pt'))

In [31]:
model._hf_peft_config_loaded = True  # silence a warning from HF trainer

trainer = transformers.Trainer(
    model=model, train_dataset=ds['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1, gradient_accumulation_steps=4,
        # note: if you want larger batch size, increase gradient_accumulation_steps
        warmup_steps=100, max_steps=200, learning_rate=2e-4, fp16=True,
        logging_steps=1, output_dir='outputs', report_to=None),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
# if you see cache warnings, set `model.config.use_cache = False` to silence them. Please re-enable for inference!

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss
1,1.138800
2,1.209900
3,0.889300
4,0.815300
5,1.230300
6,1.230600
7,1.396500
8,1.399700
9,0.862900
10,0.948700


TrainOutput(global_step=200, training_loss=1.0274927243590355, metrics={'train_runtime': 3393.1723, 'train_samples_per_second': 0.236, 'train_steps_per_second': 0.059, 'total_flos': 1.680530594955264e+16, 'train_loss': 1.0274927243590355, 'epoch': 0.02})

In [34]:
after_outputs = []
for prompt in prompts:
    after_output = test_model_output(prompt)
    after_outputs.append(after_output[3:])
    print('---------------')


Output: <s># -*- coding: utf-8 -*-
#
# Copyright (C) 2013-2014 Red Hat, Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License"); you may
# not
---------------

Output: <s>import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.patches as patches
import matplotlib.artist as artist
import matplotlib.path as path
import matplotlib.transforms as
---------------

Output: <s>from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.image as image
import matplotlib.transforms as transforms
import matplotlib.path as path
import matplotlib.patch
---------------

Output: <s>while True:
    print("Enter your name:")
    name = input()
    print("Enter your age:")
    age = int(input())
    print("Enter your gender:")
    gender = input()
    print("Enter your phon

In [36]:
# This template helps to compare generated code samples in pretty table form
# feel free to present your work in other forms

from IPython.display import HTML, display
table_template = """<table style="border:1px solid black" >
  <tr>
    <th style="text-align: center; border:1px solid black">PROMPT</th>
    <th style="text-align: center; border:1px solid black">BEFORE</th>
    <th style="text-align: center; border:1px solid black">AFTER</th>
  </tr>
{}
</table>"""

row_template = '''  <tr>
    <td style="width:20%; border:1px solid black"><pre align="left">`{}`</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
  </tr>'''

rows = []

for i, prompt in enumerate(prompts):
    # replace placeholders in the format() arguments
    rows.append(row_template.format(prompt, before_outputs[i][3:], after_outputs[i]))

display(HTML(table_template.format('\n'.join(rows))))

PROMPT,BEFORE,AFTER
``,▶▶ 2019-2020 ◀◀ The 2019-2020 school year is here! We are so excited to see what God has in store for us this year. We are looking forward to another year of growth and learning.,"# -*- coding: utf-8 -*- # # Copyright (C) 2013-2014 Red Hat, Inc. # # Licensed under the Apache License, Version 2.0 (the ""License""); you may # not"
`import`,import Foundation public extension NSURL { public var absoluteString: String { return String(cString: CFBundleGetBundleWithURL(self).UTF8String) } }package com.google.gerrit.server.restapi; import static javax.,import numpy as np import matplotlib.pyplot as plt import matplotlib.image as mpimg import matplotlib.cm as cm import matplotlib.colors as colors import matplotlib.patches as patches import matplotlib.artist as artist import matplotlib.path as path import matplotlib.transforms as
`from`,from __future__ import absolute_import from __future__ import division from __future__ import print_function import os import sys from absl import flags from tensorflow.python import pywrap_tensorflow from tensorflow.python.eager import context from,from __future__ import print_function import numpy as np import matplotlib.pyplot as plt import matplotlib.cm as cm import matplotlib.colors as colors import matplotlib.image as image import matplotlib.transforms as transforms import matplotlib.path as path import matplotlib.patch
`while`,"while(1) while(1) { // do something } \end{code} Comment: This is not the same as the OP's code. Comment: @Jeffrey: It's the same as the OP's code, except that it's","while True: print(""Enter your name:"") name = input() print(""Enter your age:"") age = int(input()) print(""Enter your gender:"") gender = input() print(""Enter your phone number:"") phone = input"
`try`,try to find the best solution for your needs. We are a team of professionals with a long experience in the field of web development. We are a team of professionals with a long experience in the field of web development. We are a team of professionals with a long experience in the field of web development,try: import json except ImportError: import simplejson as json from django.db import models from django.contrib.auth.models import User from django.contrib.auth.models import Group from django.contrib.auth.models import Permission from django.contrib
`if`,"if ( !window.atmosphere ) { window.atmosphere = {}; } (function () { var o = atmosphere.util, atmosphere = atmosphere.atmosphere = function () { var _isClosed = false, _isOpen",if __name__ == '__main__': import sys import os import argparse import numpy as np import matplotlib.pyplot as plt import matplotlib.image as mpimg import matplotlib.cm as cm import matplotlib.colors as colors
`for`,for the 2019-2020 school year. The application process for the 2019-2020 school year is now open. The application process for the 2019-2020 school year is now open. Please click here to apply.,"for i in range(1, 10): print(i) for j in range(1, 10): print(j) for k in range(1, 10): print(k) for l in range(1,"
`torch`,"torchbearer 2017-05-18 19:55:25 UTC #1 I’m a newbie to the world of RPGs, and I’m looking for a game that I can play with my wife. We’re both in our","torch.setdefaulttensortype(torch.FloatTensor) from torch.autograd import Variable from torch.nn import Functional, Conv, BatchNorm, MaxPool, Dropout, Activation, Linear, Softsign, Softplus, So"
`include`,include_once(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname(dirname,include_recipe 'chef-client::default' # Install the chef-client cookbook include_recipe 'chef-client::install' # Install the chef-client cookbook include_recipe 'chef-client::install' # Install the chef-


If you reach this: congratulations! you've completed everything in this practice session.

If you want to dig deeper, try to implement prompt-tuning (for bonus points!).
You can read more about prompt tuning variants in paper [1](https://arxiv.org/abs/2104.08691) or paper [2](https://arxiv.org/abs/2101.00190). Both versions can be implemented by passing trainable prompts as `model.forward(..., past_key_values=your_prompts)`.



### Read more

* How post-training quantization works: https://arxiv.org/abs/2208.07339
* An overview of running large models: https://huggingface.co/docs/accelerate/package_reference/big_modeling
* A general library for different adapter types: https://adapterhub.ml/


### [extra info] Running other models.

This notebook's code can run with other models of similar size, such as [Falcon-7B](https://huggingface.co/tiiuae/falcon-7b), [OPT-6.7B](https://huggingface.co/facebook/opt-6.7b) or [BLOOM-7.1B](https://huggingface.co/bigscience/bloom-7b1). However, they will require minor code tweaks:
1. change the model name in `AutoModelForCausalLM.from_pretrained()` __and__ `AutoTokenizer`
2. In the prompt tuning code, change `model.model.embed_tokens` to refer to the target model's word embeddings. Simply `print(model)` to navigate to them.
3. Change code to add Lora layers - specifically where you what the transformer block components, since those components now have different names.